# Final Assignment



## 1. Proleem description
We cluster the neighbourhoods in Amsterdam in order to compare how neighbourhoods are similar to each other.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import json
print('Libraries imported.')

Libraries imported.


## 2. Data Description

## 2.1 Geodata
Data obtain from two sources: Municitple of Amsterdam and FourSquare Location data.

In [3]:

#amsterdam_data = pd.read_csv('GEBIED_BUURTCOMBINATIES.csv',delimiter=';')  # Amsterdam divided into 99 combined-boroughs
amsterdam_data = pd.read_csv('GEBIED_BUURTEN.csv',delimiter=';')  # greater granularity for the boroughs up to 400
amsterdam_neighborhood = pd.read_excel('Stadsdeel.xlsx')

In [4]:

amsterdam_data.head()

,OBJECTNUMMER,Buurt_code,Buurt,Buurtcombinatie_code,Stadsdeel_code,Opp_m2,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 10
0,1,F81d,Calandlaan/Lelylaan,F81,F,275360.0,"POLYGON((4.800801 52.355175,4.809055 52.356842...","POLYGON((52.355175 4.800801,52.356842 4.809055...",4.809697,52.355708,NaN
1,2,F81e,Osdorp Zuidoost,F81,F,519366.0,"POLYGON((4.818583 52.357519,4.818622 52.356295...","POLYGON((52.357519 4.818583,52.356295 4.818622...",4.811344,52.353736,NaN
2,3,F82a,Osdorp Midden Noord,F82,F,215541.0,"POLYGON((4.786657 52.362712,4.795326 52.364434...","POLYGON((52.362712 4.786657,52.364434 4.795326...",4.791792,52.362078,NaN
3,4,F82b,Osdorp Midden Zuid,F82,F,258379.0,"POLYGON((4.788293 52.359736,4.796917 52.36148,...","POLYGON((52.359736 4.788293,52.36148 4.796917,...",4.793781,52.358838,NaN
4,5,F82c,Zuidwestkwadrant Osdorp Noord,F82,F,240774.0,"POLYGON((4.790209 52.356207,4.799258 52.358027...","POLYGON((52.356207 4.790209,52.358027 4.799258...",4.795597,52.355523,NaN


In [5]:
amsterdam_neighborhood

,Stadsdeel_code,Stadsdeel
0,A,Centrum
1,E,West (incl. Westpoort)
2,F,Nieuw-West
3,K,Zuid
4,M,Oost
5,N,Noord
6,T,Zuidoost


In [6]:
amsterdam_data = pd.merge(amsterdam_data,amsterdam_neighborhood,on='Stadsdeel_code')

In [7]:
amsterdam_data.head()

,OBJECTNUMMER,Buurt_code,Buurt,Buurtcombinatie_code,Stadsdeel_code,Opp_m2,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 10,Stadsdeel
0,1,F81d,Calandlaan/Lelylaan,F81,F,275360.0,"POLYGON((4.800801 52.355175,4.809055 52.356842...","POLYGON((52.355175 4.800801,52.356842 4.809055...",4.809697,52.355708,NaN,Nieuw-West
1,2,F81e,Osdorp Zuidoost,F81,F,519366.0,"POLYGON((4.818583 52.357519,4.818622 52.356295...","POLYGON((52.357519 4.818583,52.356295 4.818622...",4.811344,52.353736,NaN,Nieuw-West
2,3,F82a,Osdorp Midden Noord,F82,F,215541.0,"POLYGON((4.786657 52.362712,4.795326 52.364434...","POLYGON((52.362712 4.786657,52.364434 4.795326...",4.791792,52.362078,NaN,Nieuw-West
3,4,F82b,Osdorp Midden Zuid,F82,F,258379.0,"POLYGON((4.788293 52.359736,4.796917 52.36148,...","POLYGON((52.359736 4.788293,52.36148 4.796917,...",4.793781,52.358838,NaN,Nieuw-West
4,5,F82c,Zuidwestkwadrant Osdorp Noord,F82,F,240774.0,"POLYGON((4.790209 52.356207,4.799258 52.358027...","POLYGON((52.356207 4.790209,52.358027 4.799258...",4.795597,52.355523,NaN,Nieuw-West


In [8]:
amsterdam_data['LNG'].iloc[0]

4.8096969000000005

In [9]:
address = ' Amsterdam, North Holland, Netherlands'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Amsterdam City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Amsterdam City are 52.3727598, 4.8936041.


In [10]:
# create map of New York using latitude and longitude values
map_amsterdam = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(amsterdam_data['LAT'], amsterdam_data['LNG'], amsterdam_data['Buurt'], amsterdam_data['Stadsdeel']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
    
map_amsterdam

In [11]:
oost_data = amsterdam_data[amsterdam_data['Stadsdeel'] == 'Oost'].reset_index(drop=True)
oost_data.head()

,OBJECTNUMMER,Buurt_code,Buurt,Buurtcombinatie_code,Stadsdeel_code,Opp_m2,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 10,Stadsdeel
0,147,M57b,Nieuwe Oosterbegraafplaats,M57,M,394066.0,"POLYGON((4.932628 52.343083,4.932976 52.343275...","POLYGON((52.343083 4.932628,52.343275 4.932976...",4.938974,52.343651,NaN,Oost
1,255,M27a,Swammerdambuurt,M27,M,140917.0,"POLYGON((4.904633 52.359205,4.905163 52.359341...","POLYGON((52.359205 4.904633,52.359341 4.905163...",4.907761,52.357972,NaN,Oost
2,256,M27b,Weesperzijde Midden/Zuid,M27,M,236435.0,"POLYGON((4.906552 52.355377,4.907559 52.355579...","POLYGON((52.355377 4.906552,52.355579 4.907559...",4.911531,52.352242,NaN,Oost
3,257,M27c,Parooldriehoek,M27,M,69352.2,"POLYGON((4.911329 52.355629,4.912022 52.355798...","POLYGON((52.355629 4.911329,52.355798 4.912022...",4.913618,52.353434,NaN,Oost
4,258,M28a,Oosterparkbuurt Noordwest,M28,M,220443.0,"POLYGON((4.908321 52.360582,4.908548 52.360632...","POLYGON((52.360582 4.908321,52.360632 4.908548...",4.912888,52.359078,NaN,Oost


In [12]:
address = 'Amsterdam oost'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Amsterdam Oost are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Amsterdam Oost are 52.3566078, 4.9305774.


In [13]:
# create map using latitude and longitude values
map_amsterdamoost = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(oost_data['LAT'], oost_data['LNG'], oost_data['Buurt']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdamoost)  
    
map_amsterdamoost


## 2.2 Venue data from Foursquare API


In [14]:
oost_data.loc[0, 'Buurt']

'Nieuwe Oosterbegraafplaats'

In [15]:
neighborhood_latitude = oost_data.loc[0, 'LAT'] # neighborhood latitude value
neighborhood_longitude = oost_data.loc[0, 'LNG'] # neighborhood longitude value

neighborhood_name = oost_data.loc[0, 'Buurt'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Nieuwe Oosterbegraafplaats are 52.34365145, 4.9389737.


In [16]:
CLIENT_ID = 'AKJ35FLMPQ3JL1OR0FHW0T10M3QOXI1MHXURU1RZWXX1CXEE' # your Foursquare ID
CLIENT_SECRET = '4PMZZ5VUWIHK1JEKMVOF02JXI534H4BI54O1ITSEP4ZY0YB1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AKJ35FLMPQ3JL1OR0FHW0T10M3QOXI1MHXURU1RZWXX1CXEE
CLIENT_SECRET:4PMZZ5VUWIHK1JEKMVOF02JXI534H4BI54O1ITSEP4ZY0YB1


In [17]:
# type your answer here
# create URL
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AKJ35FLMPQ3JL1OR0FHW0T10M3QOXI1MHXURU1RZWXX1CXEE&client_secret=4PMZZ5VUWIHK1JEKMVOF02JXI534H4BI54O1ITSEP4ZY0YB1&v=20180605&ll=52.34365145,4.9389737&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607c1985214fbf5eec851faa'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Betondorp',
  'headerFullLocation': 'Betondorp, Amsterdam',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 52.348151454500005,
    'lng': 4.946325845333216},
   'sw': {'lat': 52.3391514455, 'lng': 4.931621554666784}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a2706f7f964a520868c1fe3',
       'name': 'Tot Zover (Nederlands Uitvaartmuseum)',
       'location': {'address': 'Kruislaan 124',
        'lat': 52.34606304054166,
        'lng': 4.938405744242209,
        'labeledLatLngs': [{'label': 'display',
       

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()


7 venues were returned by Foursquare.
ipykernel_launcher:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Tot Zover (Nederlands Uitvaartmuseum),Museum,52.346063,4.938406
1,Arboretum De Nieuwe Ooster,Park,52.346913,4.938474
2,Robert kochplantsoen,Park,52.347380,4.937297
3,Tramhalte Kruislaan,Tram Station,52.347601,4.940566
4,De Lekkernij,Bakery,52.340494,4.943545


## 2.3 Venue data in batches


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
oost_venues = getNearbyVenues(names = oost_data['Buurt'],
                              latitudes=oost_data['LAT'],
                              longitudes=oost_data['LNG']
                              )

Nieuwe Oosterbegraafplaats
Swammerdambuurt
Weesperzijde Midden/Zuid
Parooldriehoek
Oosterparkbuurt Noordwest
Oosterpark
Oosterparkbuurt Zuidoost
Oosterparkbuurt Zuidwest
Dapperbuurt Noord
Dapperbuurt Zuid
Transvaalbuurt West
Transvaalbuurt Oost
Noordwestkwadrant Indische buurt Noord
Noordwestkwadrant Indische buurt Zuid
Zuidwestkwadrant Indische buurt
Noordoostkwadrant Indische buurt
Zuidoostkwadrant Indische buurt
Zeeburgerdijk Oost
Flevopark
Oostelijke Handelskade
Java-eiland
KNSM-eiland
Sporenburg
Borneo
Entrepot-Noordwest
Architectenbuurt
Bedrijvengebied Veelaan
Zeeburgereiland Noordwest
Zeeburgereiland Zuidoost
Steigereiland Noord
Steigereiland Zuid
Haveneiland Zuidwest/Rieteiland West
Rieteiland Oost
De Eenhoorn
Don Bosco
Frankendael
De Wetbuurt
Tuindorp Frankendael
Linnaeusparkbuurt
Middenmeer Noord
Middenmeer Zuid
Sportpark Middenmeer Zuid
Sportpark Middenmeer Noord
Science Park Noord
Haveneiland Noordwest
RI Oost terrein
Zeeburgereiland Noordoost
Van der Kunbuurt
Haveneiland N

In [25]:
print(oost_venues.shape)
oost_venues.head()

(1666, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Nieuwe Oosterbegraafplaats,52.343651,4.938974,Tot Zover (Nederlands Uitvaartmuseum),52.346063,4.938406,Museum
1,Nieuwe Oosterbegraafplaats,52.343651,4.938974,Arboretum De Nieuwe Ooster,52.346913,4.938474,Park
2,Nieuwe Oosterbegraafplaats,52.343651,4.938974,Robert kochplantsoen,52.347380,4.937297,Park
3,Nieuwe Oosterbegraafplaats,52.343651,4.938974,Tramhalte Kruislaan,52.347601,4.940566,Tram Station
4,Nieuwe Oosterbegraafplaats,52.343651,4.938974,De Lekkernij,52.340494,4.943545,Bakery


In [26]:
oost_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amstelglorie,4,4,4,4,4,4
Amstelkwartier Noord,19,19,19,19,19,19
Amstelkwartier West,22,22,22,22,22,22
Amstelkwartier Zuid,25,25,25,25,25,25
Architectenbuurt,8,8,8,8,8,8
Bedrijvengebied Cruquiusweg,6,6,6,6,6,6
Bedrijvengebied Veelaan,35,35,35,35,35,35
Bedrijvengebied Zeeburgerkade,27,27,27,27,27,27
Betondorp,2,2,2,2,2,2


In [27]:
print('There are {} uniques categories.'.format(len(oost_venues['Venue Category'].unique())))

There are 191 uniques categories.


In [28]:
oost_venues.to_csv('Amsterdam_Oost.csv')

## 3. Analyze Each Neighborhood


The similiarity of the neighthoods are determined by K-mean algorithme, features used are venues within a certain radius around the geodata point given.


In [29]:
# one hot encoding
oost_onehot = pd.get_dummies(oost_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
oost_onehot['Neighborhood'] = oost_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [oost_onehot.columns[-1]] + list(oost_onehot.columns[:-1])
oost_onehot = oost_onehot[fixed_columns]

print(oost_onehot.shape)
oost_onehot.head()

(1666, 191)


,Zoo Exhibit,Advertising Agency,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Campground,Canal,Canal Lock,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Science Building,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Drugstore,Dutch Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Truck,French Restaurant,Friterie,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Korean Restaurant,Lake,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Moving Target,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Restaurant,Rock Club,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Waterfront,Windmill,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Nieuwe Oosterbegraafplaats,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nieuwe Oosterbegraafplaats,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nieuwe Oosterbegraafplaats,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Now we will group the data by neighborhood

In [30]:
oost_grouped = oost_onehot.groupby('Neighborhood').mean().reset_index()
print(oost_grouped.shape)
oost_grouped

(76, 191)


,Neighborhood,Zoo Exhibit,Advertising Agency,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Campground,Canal,Canal Lock,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Science Building,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Drugstore,Dutch Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Truck,French Restaurant,Friterie,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Korean Restaurant,Lake,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Moving Target,Museum,Music School,Music Store,Music Venue,Nightclub,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Restaurant,Rock Club,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Waterfront,Windmill,Wine Bar,Yoga Studio
0,Amstelglorie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

Generate top five most common venues

In [31]:
num_top_venues = 5

for hood in oost_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = oost_grouped[oost_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amstelglorie----
                       venue  freq
0       Gym / Fitness Center   0.5
1         Athletics & Sports   0.5
2                Zoo Exhibit   0.0
3                  Nightclub   0.0
4  Middle Eastern Restaurant   0.0


----Amstelkwartier Noord----
              venue  freq
0             Hotel  0.11
1  Indoor Play Area  0.05
2         Beach Bar  0.05
3              Park  0.05
4        Restaurant  0.05


----Amstelkwartier West----
                  venue  freq
0                 Hotel  0.14
1  Gym / Fitness Center  0.14
2            Restaurant  0.14
3    Athletics & Sports  0.09
4          Dance Studio  0.05


----Amstelkwartier Zuid----
          venue  freq
0         Hotel  0.20
1    Restaurant  0.12
2  Dance Studio  0.08
3           Bar  0.04
4   Coffee Shop  0.04


----Architectenbuurt----
             venue  freq
0              Bar  0.12
1       Playground  0.12
2  Harbor / Marina  0.12
3   Farmers Market  0.12
4              Gym  0.12


----Bedrijvengebied Cruquiusweg

Store in dataframe

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = oost_grouped['Neighborhood']

for ind in np.arange(oost_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(oost_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amstelglorie,Gym / Fitness Center,Athletics & Sports,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Farm,Ethiopian Restaurant,Eastern European Restaurant,Dutch Restaurant
1,Amstelkwartier Noord,Hotel,Cocktail Bar,Indoor Play Area,Sports Club,Beach Bar,Café,Boat or Ferry,Furniture / Home Store,Dutch Restaurant,Italian Restaurant
2,Amstelkwartier West,Gym / Fitness Center,Hotel,Restaurant,Athletics & Sports,Auto Garage,Breakfast Spot,Sandwich Place,Boat or Ferry,Food Truck,Beach Bar
3,Amstelkwartier Zuid,Hotel,Restaurant,Dance Studio,Indoor Play Area,Food Truck,Beach Bar,Bistro,Bar,Men's Store,Furniture / Home Store
4,Architectenbuurt,Harbor / Marina,Café,Farmers Market,Gym,Playground,Lounge,Dutch Restaurant,Bar,Farm,Ethiopian Restaurant


## 4. Cluster the neighbourhood


In [34]:
kclusters = 5

oost_grouped_clustering = oost_grouped.drop('Neighborhood',1)

# run k-mean clustering
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(oost_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 0, 0, 0, 4, 4, 0, 0, 4, 4], dtype=int32)

create dataframes that includes the cluster as well as the top 10 venues for each neighborhood

In [35]:
# add cluster labels
neighborhoods_venues_sorted.insert(1,'Cluster Labels', kmeans.labels_)
oost_merged = oost_data
oost_merged = oost_merged.rename(columns={"Buurt": "Neighborhood"})
# merge oost_grouped with oost_data to add lat and lng for each neighbourhood
oost_merged = oost_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
oost_merged

,OBJECTNUMMER,Buurt_code,Neighborhood,Buurtcombinatie_code,Stadsdeel_code,Opp_m2,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 10,Stadsdeel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,147,M57b,Nieuwe Oosterbegraafplaats,M57,M,394066.0,"POLYGON((4.932628 52.343083,4.932976 52.343275...","POLYGON((52.343083 4.932628,52.343275 4.932976...",4.938974,52.343651,NaN,Oost,4.0,Park,Tram Station,Museum,Bakery,Bus Stop,Yoga Studio,Farm,Ethiopian Restaurant,Eastern European Restaurant,Dutch Restaurant
1,255,M27a,Swammerdambuurt,M27,M,140917.0,"POLYGON((4.904633 52.359205,4.905163 52.359341...","POLYGON((52.359205 4.904633,52.359341 4.905163...",4.907761,52.357972,NaN,Oost,0.0,Hotel,Italian Restaurant,Yoga Studio,Café,Coffee Shop,Middle Eastern Restaurant,Restaurant,Bakery,Modern European Restaurant,Marijuana Dispensary
2,256,M27b,Weesperzijde Midden/Zuid,M27,M,236435.0,"POLYGON((4.906552 52.355377,4.907559 52.355579...","POLYGON((52.355377 4.906552,52.355579 4.907559...",4.911531,52.352242,NaN,Oost,0.0,Café,Pizza Place,Hotel Bar,Hotel,Bar,Breakfast Spot,African Restaurant,Restaurant,Bakery,Pool
3,257,M27c,Parooldriehoek,M27,M,69352.2,"POLYGON((4.911329 52.355629,4.912022 52.355798...","POLYGON((52.355629 4.911329,52.355798 4.912022...",4.913618,52.353434,NaN,Oost,0.0,Breakfast Spot,Restaurant,Hotel,Supermarket,Bar,Bakery,Café,Hotel Bar,Pizza Place,Italian Restaurant
4,258,M28a,Oosterparkbuurt Noordwest,M28,M,220443.0,"POLYGON((4.908321 52.360582,4.908548 52.360632...","POLYGON((52.360582 4.908321,52.360632 4.908548...",4.912888,52.359078,NaN,Oost,0.0,Hotel,Coffee Shop,Italian Restaurant,Bar,Bakery,Gym / Fitness Center,Theater,Café,Playground,Liquor Store
5,259,M28b,Oosterpark,M28,M,253968.0,"POLYGON((4.91508 52.361779,4.91577 52.36197,4....","POLYGON((52.361779 4.91508,52.36197 4.91577,52...",4.920192,52.360886,NaN,Oost,0.0,Zoo Exhibit,Bar,Italian Restaurant,Café,Coffee Shop,Hotel,Gift Shop,Park,Organic Grocery,Museum
6,260,M28c,Oosterparkbuurt Zuidoost,M28,M,157594.0,"POLYGON((4.917445 52.358036,4.917773 52.358134...","POLYGON((52.358036 4.917445,52.358134 4.917773...",4.922073,52.357827,NaN,Oost,0.0,Coffee Shop,Italian Restaurant,Sporting Goods Shop,Bar,Café,Turkish Restaurant,Liquor Store,Asian Restaurant,Organic Grocery,Spiritual Center
7,261,M28d,Oosterparkbuurt Zuidwest,M28,M,121888.0,"POLYGON((4.910879 52.356392,4.917445 52.358036...","POLYGON((52.356392 4.910879,52.358036 4.917445...",4.915037,52.356167,NaN,Oost,0.0,Italian Restaurant,Coffee Shop,Breakfast Spot,Supermarket,Hotel Bar,Café,Bakery,Asian Restaurant,Restaurant,Gym / Fitness Center
8,262,M29a,Dapperbuurt Noord,M29,M,211196.0,"POLYGON((4.922313 52.363752,4.924779 52.365241...","POLYGON((52.363752 4.922313,52.365241 4.924779...",4.927210,52.364237,NaN,Oost,0.0,Bar,Café,Zoo Exhibit,Middle Eastern Restaurant,Restaurant,Italian Restaurant,Market,Supermarket,Turkish Restaurant,Modern European Restaurant
9,263,M29b,Dapperbuurt Zuid,M29,M,174086.0,"POLYGON((4.923984 52.361924,4.930343 52.363279...","POLYGON((52.361924 4.923984,52.363279 4.930343...",4.927701,52.360610,NaN,Oost,0.0,Turkish Restaurant,Café,Bar,Coffee Shop,Market,Park,Museum,Caribbean Restaurant,Pizza Place,Organic Grocery


creat map and visualise the clusters

In [37]:

# oost_merged['Cluster Labels'].fillna(5,inplace=True) # we have some NaN from area which will be buildt in the future Buiteneiland in Ijburg, we add more cluster
oost_merged['Cluster Labels'].fillna(kclusters+1,inplace=True)
oost_merged['Cluster Labels']= oost_merged['Cluster Labels'].astype('int64')


In [39]:
oost_merged.head()

,OBJECTNUMMER,Buurt_code,Neighborhood,Buurtcombinatie_code,Stadsdeel_code,Opp_m2,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 10,Stadsdeel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,147,M57b,Nieuwe Oosterbegraafplaats,M57,M,394066.0,"POLYGON((4.932628 52.343083,4.932976 52.343275...","POLYGON((52.343083 4.932628,52.343275 4.932976...",4.938974,52.343651,NaN,Oost,4,Park,Tram Station,Museum,Bakery,Bus Stop,Yoga Studio,Farm,Ethiopian Restaurant,Eastern European Restaurant,Dutch Restaurant
1,255,M27a,Swammerdambuurt,M27,M,140917.0,"POLYGON((4.904633 52.359205,4.905163 52.359341...","POLYGON((52.359205 4.904633,52.359341 4.905163...",4.907761,52.357972,NaN,Oost,0,Hotel,Italian Restaurant,Yoga Studio,Café,Coffee Shop,Middle Eastern Restaurant,Restaurant,Bakery,Modern European Restaurant,Marijuana Dispensary
2,256,M27b,Weesperzijde Midden/Zuid,M27,M,236435.0,"POLYGON((4.906552 52.355377,4.907559 52.355579...","POLYGON((52.355377 4.906552,52.355579 4.907559...",4.911531,52.352242,NaN,Oost,0,Café,Pizza Place,Hotel Bar,Hotel,Bar,Breakfast Spot,African Restaurant,Restaurant,Bakery,Pool
3,257,M27c,Parooldriehoek,M27,M,69352.2,"POLYGON((4.911329 52.355629,4.912022 52.355798...","POLYGON((52.355629 4.911329,52.355798 4.912022...",4.913618,52.353434,NaN,Oost,0,Breakfast Spot,Restaurant,Hotel,Supermarket,Bar,Bakery,Café,Hotel Bar,Pizza Place,Italian Restaurant
4,258,M28a,Oosterparkbuurt Noordwest,M28,M,220443.0,"POLYGON((4.908321 52.360582,4.908548 52.360632...","POLYGON((52.360582 4.908321,52.360632 4.908548...",4.912888,52.359078,NaN,Oost,0,Hotel,Coffee Shop,Italian Restaurant,Bar,Bakery,Gym / Fitness Center,Theater,Café,Playground,Liquor Store


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# adjust the cluster + 1 for NaN
kclusters = kclusters +1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(oost_merged['LAT'], oost_merged['LNG'], oost_merged['Neighborhood'], oost_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters